In [1]:
addprocs(5);

## Fonctions

* __remotecall(f, id::Integer, args...; kwargs...)__ : Appelez une fonction de manière asynchrone sur les arguments donnés sur le processus spécifié. Renvoie un Futur. Les arguments suivant, le cas échéant, sont transmis à la fonction.
* __fetch(c::Channel)__ : Attend et obtient le premier article disponible du Channel. Ne supprime pas l'élément.
* __remotecall_fetch(f, id::Integer, args...; kwargs...)__ : Performe un remotecall() et un fetch() immédiatement.
* __pmap(f, c)__ : Transforme une collection c en appliquant une fonction f sur chaque element en utilisant les processus disponibles.

In [2]:
r = remotecall(rand, 2, 2, 2)
r_f = fetch(r)

2×2 Array{Float64,2}:
 0.150686   0.761927
 0.0407421  0.462775

In [3]:
remotecall_fetch(getindex, 2, r_f, 1, 1)

0.15068588960312157

In [4]:
M = Matrix{Float64}[rand(1000,1000) for i = 1:10];
pmap(mean, M)

10-element Array{Float64,1}:
 0.499893
 0.499984
 0.500502
 0.49973 
 0.500109
 0.499807
 0.500259
 0.499943
 0.499727
 0.500227

## Macros
* __@spawnat__ : Accepte deux arguments, un processus p et une expression. Une Closure est créé autour de l'expression et est exécuté async sur le processus p. Retourne un Future vers le résultat.
* __@everywhere__ : Exécute une expression dans tous les processus.
* __@parallel__ : Exécute un for loop sur différent processus en distribuant chaque loop. peut etre accompagné d'une fonction de réduction.

In [5]:
s = @spawnat 2 2 .* r_f
fetch(s)

2×2 Array{Float64,2}:
 0.301372   1.52385 
 0.0814843  0.925551

In [6]:
@everywhere function f(x)
    x^2
end

@everywhere Z = [1 2 3; 1 2 3; 1 2 3]
Z

3×3 Array{Int64,2}:
 1  2  3
 1  2  3
 1  2  3

In [7]:
nheads = @parallel (+) for i = 1:200000000
    Int(rand(Bool))
end

99994991

In [8]:
a = zeros(10)
@parallel for i = 1:10
    a[i] = i
end

a

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [9]:
a = SharedArray{Float64}(10)
@parallel for i = 1:10
    a[i] = i
end

a

10-element SharedArray{Float64,1}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0

In [10]:
# read-only
a = randn(1000)
@parallel (+) for i = 1:10
    f(a[rand(1:end)])
end

10.75143566779041

# Communication
* __Channel{T}(sz)__ : Construit un Channel avec un tampon interne qui permet de contenir un nombre sz d'items de type T. Insérer avec put! et retirer avec take!. Si l'un ou l'autre est bloqué, il attend.

In [11]:
jobs = Channel{Int}(32);
results = Channel{Tuple}(32);

function do_work()
   for job_id in jobs
       exec_time = rand()
       sleep(exec_time)               
       put!(results, (job_id, exec_time))
   end
end;

function make_jobs(n)
   for i in 1:n
       put!(jobs, i)
   end
end;

n = 12;
@schedule make_jobs(n);

for i in 1:4 
   @schedule do_work()
end

@elapsed while n > 0
   job_id, exec_time = take!(results)
   println("$job_id terminé en $(round(exec_time,2)) secondes")
   n = n - 1
end;

4 terminé en 0.46 secondes
2 terminé en 0.64 secondes
1 terminé en 0.7 secondes
3 terminé en 0.7 secondes
5 terminé en 0.25 secondes
6 terminé en 0.29 secondes
8 terminé en 0.37 secondes
11 terminé en 0.25 secondes
9 terminé en 0.66 secondes
10 terminé en 0.52 secondes
7 terminé en 0.82 secondes
12 terminé en 0.35 secondes
